<a href="https://colab.research.google.com/github/MiguelRochaBE/MotorImagery-BCI/blob/master/MI_BCI_DatasetAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Conexão ao Google Colab**

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


## **Extrair base de dados do ficheiro zip**

Link: https://physionet.org/content/eegmmidb/1.0.0/

In [ ]:
!unzip -u "/content/drive/MyDrive/B(i)onitos/files.zip" -d "/content/drive/MyDrive"

unzip:  cannot find or open /content/drive/folders/1-AQ0e7zyb4ns3ZV7YGILoz95nTGsGBc8/files.zip, /content/drive/folders/1-AQ0e7zyb4ns3ZV7YGILoz95nTGsGBc8/files.zip.zip or /content/drive/folders/1-AQ0e7zyb4ns3ZV7YGILoz95nTGsGBc8/files.zip.ZIP.


## **Listar todos os ficheiros da base de dados**

In [19]:
import os

def run_fast_scandir(dir, ext):    # dir: str, ext: list
    subfolders, datafiles = [], []

    for f in os.scandir(dir):
        if f.is_dir():
            subfolders.append(f.path)
        if f.is_file():
            if os.path.splitext(f.name)[1].lower() in ext:
                datafiles.append(f.path)

    for dir in list(subfolders):
        sf, f = run_fast_scandir(dir, ext)
        subfolders.extend(sf)
        datafiles.extend(f)
    return subfolders, datafiles


subfolders, datafiles = run_fast_scandir('/content/drive/MyDrive/files', [".edf"])
#subfolders, eventfiles = run_fast_scandir('/content/drive/MyDrive/files', [".event"])

datafiles.sort()
print(subfolders[:3])
print(datafiles[:3])


['/content/drive/MyDrive/files/S066', '/content/drive/MyDrive/files/S067', '/content/drive/MyDrive/files/S068']
['/content/drive/MyDrive/files/S001/S001R01.edf', '/content/drive/MyDrive/files/S001/S001R02.edf', '/content/drive/MyDrive/files/S001/S001R03.edf']


## **Biblioteca MNE: https://mne.tools/stable/index.html**


In [5]:
!pip install mne
import mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.6 MB/s eta 0:00:00


## **Ler dados e separar/agrupar os blocos de EEG consoante o tipo/classe de movimento**

In [ ]:
def load_subject_data(subject, datafiles):
  subject_number = subject.split('files/S')[1][:2] # [1] corresponde à segunda string da string original dividida 

  trials=[] # Pré-alocação do array na memória
  for trial in datafiles:
    if subject_number in trial:
      

  
  data = mne.io.read_raw_edf(trial, preload = True) # Dados EEG para um trial específico. 'Preload=True' serve para colocar o ficheiro na memória e não apenas referenciá-lo no diretório
  
  ch_names = data.ch_names # Elétrodos
  print("\nElétrodos: ", ch_names)

  fs =  data.info.get('sfreq') # Frequência de amostragem
  annotations = data.annotations # Variável que contém os dados no tempo relativos aos blocos de movimentos executados
  
  trial_type1 = [3, 4, 7, 8, 11, 12]
  trial_type2 = [5, 6, 9, 10, 13, 14]

  '''
  Descrição dos eventos:

  Each annotation includes one of three codes (T0, T1, or T2):

    - T0 corresponds to rest
    - T1 corresponds to onset of motion (real or imagined) of the left fist (in runs 3, 4, 7, 8, 11, and 12)
                                                                  both fists (in runs 5, 6, 9, 10, 13, and 14)
    - T2 corresponds to onset of motion (real or imagined) of the right fist (in runs 3, 4, 7, 8, 11, and 12)
                                                                  both feet (in runs 5, 6, 9, 10, 13, and 14)
  '''

  # Atribuição das labels corretas aos eventos de cada trial

  trial_number = int(trial.split('.')[0][-2:]) # De 1 a 14
  print("\nTrial: ",trial_number)
  if trial_number in trial_type1:
    event_ids = {'T0': 1, 'T1': 2, 'T2': 4} # Dicionário com os ids dos eventos de acordo com a descrição a laranja ^^
    epoch_event_ids = {"rest/block": 1, "left_fist/block": 2, "both_fists/block": 4}
  elif trial_number in trial_type2:
    event_ids = {'T0': 1, 'T1': 3, 'T2': 5}
    epoch_event_ids = {"rest/block": 1, "right_fist/block": 3,"both_feet/block": 5}
  else:
    event_ids = {'T0': 1}

  events = mne.events_from_annotations(data, event_id = event_ids) # Variável eventos: Retorna 3 colunas [Indice da amostra onde começa o bloco, Trigger, Tipo de movimento (label)]
  print("\n")
  figure = mne.viz.plot_events(events[0]) # Gráfico dos eventos

  print("\nInfo dos dados: ", data.info)
  print("\nAnotações: ", annotations.description)
  print("\nEventos: ", events)
  print("\n")

  # Agrupamento dos blocos do mesmo id

  epochs = mne.Epochs(data, events[0], event_id = epoch_event_ids, baseline = None, preload = True)
  print("Informação dos blocos: ", epochs.info)
  #epochs["right_fist"].plot_image(picks=[8])

  data.plot(events=events[0])

load_subject_data(datafiles[5])


The number is in the list.
